In [1]:
cd ..

D:\J.H.LEE\05. CODING\Python\Project\tdatlib


In [2]:
import plotly.graph_objects as go
import pandas as pd
import tdatlib as tlib
from plotly.subplots import make_subplots
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important;}</style>"))

# Initialize

In [4]:
ecos, fred = tlib.macro.ecos(), tlib.macro.fred()
market = tlib.market.KR()
index = tlib.index()
index.data.period = ecos.period = fred.period = 20

tickers = pd.DataFrame(
    data=[
        dict(ticker="5046",   종류="지수", 이름="KRX은행"),
        dict(ticker="091170", 종류="ETF", 이름="KODEX은행"),
        dict(ticker="091220", 종류="ETF", 이름="TIGER은행"),
        dict(ticker="105560", 종류="주식", 이름="KB금융"),
        dict(ticker="086790", 종류="주식", 이름="하나금융지주"),
        dict(ticker="139130", 종류="주식", 이름="DGB금융지주"),
        dict(ticker="055550", 종류="주식", 이름="신한지주"),
        dict(ticker="024110", 종류="주식", 이름="기업은행"),
        dict(ticker="138930", 종류="주식", 이름="BNK금융지주"),
        dict(ticker="316140", 종류="주식", 이름="우리금융지주"),
        dict(ticker="038540", 종류="주식", 이름="상상인"),
    ]
).set_index(keys='ticker').join(other=market.icm)

stock_objs = {ticker: tlib.stock.kr(ticker=ticker) for ticker in tickers[tickers.종류 == '주식'].index}

In [5]:
# 레이아웃
layout = dict(
    plot_bgcolor='white',
    height=750,
    legend=dict(
        orientation="h",
        xanchor="right", yanchor="bottom",
        x=0.96, y=1,
    ),
)

# Data Collection

## KRX은행지수

In [10]:
# KRX 은행 지수
krx_bank = index.data.bank
krx_bank_trace = tlib.toolbox.tracer.line(krx_bank, 'KRX은행')
krx_bank_trace.lineconfig(dict(color='black', width=1))

## 수신/대출 금리 정보

In [17]:
col_table = pd.concat(
    objs={
        "수신신규 121Y002":ecos.contains('121Y002').이름.drop_duplicates(keep='first'),
        "수신잔액 121Y013":ecos.contains('121Y013').이름.drop_duplicates(keep='first'),
        "대출신규 121Y006":ecos.contains('121Y006').이름.drop_duplicates(keep='first'),
        "대출잔액 121Y015":ecos.contains('121Y015').이름.drop_duplicates(keep='first'),
    }, axis=1
)
col_table

,수신신규 121Y002,수신잔액 121Y013,대출신규 121Y006,대출잔액 121Y015
0,저축성수신,총수신(요구불예금 및 수시입출식 저축성예금 포함),대출평균 1),총대출 1)
3,순수저축성예금 1),수시입출식 저축성예금,기업대출,총대출(당좌대출 제외)
6,정기예금,저축성수신(요구불예금 및 수시입출식 저축성예금 제외),대기업대출,기업대출 2)
9,정기예금(6개월 미만) 2),저축성수신(금융채 제외),중소기업대출,대기업대출
12,정기예금(6개월~1년 미만) 2),순수저축성예금,운전자금대출,중소기업대출
15,정기예금(1~2년 미만) 2),정기예금,시설자금대출,운전자금대출
18,정기예금(2~3년 미만),정기적금,가계대출,시설자금대출
21,정기예금(3~4년 미만),상호부금,소액대출 (500만원 이하),가계대출 2)
24,정기예금(4~5년 미만),주택부금,주택담보대출,소액대출 (500만원 이하)
27,정기예금(5년 이상),시장형금융상품,예·적금담보대출,주택담보대출


In [18]:
# depo = ecos.load('121Y002', '저축성수신')
# depo = ecos.load('121Y002', '저축성수신(금융채 제외) 1)')
depo = ecos.load('121Y013', '총수신(요구불예금 및 수시입출식 저축성예금 포함)')

# loan = ecos.load('121Y006', '대출평균 1)')
# loan = ecos.load('121Y015', '총대출 1)')
loan = ecos.load('121Y015', '총대출(당좌대출 제외)')

r_depo_loan = loan / depo
d_depo_loan = loan - depo
depo_trace = tlib.toolbox.tracer.line(depo, name='수신금리')
loan_trace = tlib.toolbox.tracer.line(loan, name='대출금리')
d_depo_loan_trace = tlib.toolbox.tracer.line(d_depo_loan, name='예대출금리차')
r_depo_loan_trace = tlib.toolbox.tracer.line(r_depo_loan, name='예대출금리비')

ParserError: month must be in 1..12: 201301 present at position 99

## 기준 금리

In [ ]:
# 한국은행 기준금리
kr_std = macro.data.KR_IR
kr_std_trace = go.Scatter(
    x=kr_std.index,
    y=kr_std,
    name='한국기준금리',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

# 미국 기준금리
us_std = macro.data.US_IR
us_std_trace = go.Scatter(
    x=us_std.index,
    y=us_std,
    name='미국기준금리',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

## 2년물 금리

In [ ]:
df = macro.data.ecos('817Y002')
print(len(df.columns), df.columns)

In [ ]:
# 한국채 2년물 금리
# kr_2yt = macro.data.KR_2Y_TY
kr_2yt = macro.data.ecos('817Y002')['국고채(1년)']
kr_2yt_trace = go.Scatter(
    x=kr_2yt.index,
    y=kr_2yt,
    name='한국국고채2년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

# 미국채 2년물 금리
us_2yt = macro.data.US_2Y_TY
us_2yt_trace = go.Scatter(
    x=us_2yt.index,
    y=us_2yt,
    name='미국국고채2년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

## 10년물 금리

In [ ]:
# 한국채 10년물 금리
kr_10yt = macro.data.KR_10Y_TY
kr_10yt_trace = go.Scatter(
    x=kr_10yt.index,
    y=kr_10yt,
    name='한국국고채10년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

# 미국채 10년물 금리
us_10yt = macro.data.US_10Y_TY
us_10yt_trace = go.Scatter(
    x=us_10yt.index,
    y=us_10yt,
    name='미국국고채10년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

## 원-달러 환율

In [ ]:
# 원-달러 환율
exchange = macro.data.KRW_USD_exchange
exchange_trace = go.Candlestick(
    name='원-달러 환율',
    x=exchange.index,
    open=exchange.시가,
    high=exchange.고가,
    low=exchange.저가,
    close=exchange.종가,
    visible=True,
    showlegend=True,
    increasing_line=dict(color='red'),
    decreasing_line=dict(color='royalblue'),
    xhoverformat='%Y/%m/%d',
    yhoverformat='.2f',
)

# BM & Products

- 은행 사업 구조 및 최대 매출처 확인
- 주요 매출처
  * 유가 증권 평가 및 처분 이익<br>
    : 1년 또는 1영업주기 내 현금화할 의도가 있는 증권을 유가 증권이라고 한다.<br>
    : 증권은 CD, CP 등 단기 증서 및 보통주/우선주의 주식과 사채, 어음 등 채무 증권을 말한다.<br>
    : 이러한 유가 증권을 취득 가액보다 높은 이익으로 처분할 때 발생하는 이익 및 보유 증권의 평가액을 매출에 반영한다.<br>
  * 신탁 업무 운용 수익
    : 운용

## 대표 시중은행 매출 구조

In [ ]:
target = tickers.sort_values(by='시가총액', ascending=False)[:4]
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "domain"}, {"type": "domain"}],
        [{"type": "domain"}, {"type": "domain"}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    products = obj.fnguide.Products
    fig.add_trace(
        trace = go.Pie(
            name='Product',
            labels=products.index,
            values=products,
            visible=True,
            showlegend=True,
            automargin=True,
            textfont=dict(color='white'),
            textinfo='label+percent',
            insidetextorientation='radial',
            hoverinfo='label+percent',
        ), row = n // 2 + 1, col = n % 2 + 1
    )

fig.update_layout(layout)
fig.update_layout(    
    title=f'<b>은행주</b> 주요 매출',
    legend=dict(y=1.08)
)

for n, annotation in enumerate(fig['layout']['annotations']):
    annotation['x'] = 0 + 0.55 * (n % 2)
    annotation['xanchor'] = 'center'
    annotation['xref'] = 'paper'
fig.show()

## 매출 비중

In [ ]:
p = pd.concat(objs=[stock_objs[ticker].fnguide.Products for n, ticker in enumerate(target.index)], axis=1).T.sum()
portion = round(100 * p / p.sum(), 2).sort_values()

fig = go.Figure()
fig.add_trace(
    trace=go.Bar(
        x=portion, y=portion.index, orientation='h',
        texttemplate='%{x:.2f}%', textposition='inside', hoverinfo='skip'
    )
)
fig.update_layout(
    title='시중은행 주요 매출처 비중', plot_bgcolor='white', height=500,
    xaxis=dict(title='비중[%]', showgrid=True, gridcolor='lightgrey', showline=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Macro-Relation

## KRX은행 vs 금리

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(trace=krx_bank_trace, secondary_y=False)
# fig.add_trace(trace=depo_trace, secondary_y=True)
# fig.add_trace(trace=loan_trace, secondary_y=True)
# fig.add_trace(trace=l_d_diff_trace, secondary_y=True)
# fig.add_trace(trace=l_d_ratio_trace, secondary_y=True)
fig.add_trace(trace=kr_std_trace, secondary_y=True)
fig.add_trace(trace=us_std_trace, secondary_y=True)
fig.add_trace(trace=kr_2yt_trace, secondary_y=True)
fig.add_trace(trace=us_2yt_trace, secondary_y=True)
# fig.add_trace(trace=us_10yt_trace, secondary_y=True)

add_recession(fig)
fig.update_layout(layout)
fig.update_layout(
    title=f'<b>KRX은행지수</b>와 금리',
    xaxis=dict(
        title='날짜',
        tickformat='%Y/%m/%d',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        title='지수',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='이율[%]',
        showticklabels=True
    )
)

## 원-달러 환율과 KRX은행지수

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(trace=krx_bank_trace, secondary_y=False)
fig.add_trace(trace=exchange_trace, secondary_y=True)
add_recession(fig)
fig.update_layout(layout)
fig.update_layout(
    title=f'<b>KRX은행지수</b>와 원-달러 환율',
    xaxis=dict(
        title='날짜',
        tickformat='%Y/%m/%d',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey',
    ),
    yaxis=dict(
        title='지수',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='KRW',
        showticklabels=True
    ),
    xaxis_rangeslider=dict(visible=False)
)

# Foreigners

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}],
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    foreigner = obj.fnguide.Foreigner['3Y'].dropna()
    fig.add_trace(
        trace = go.Scatter(
            name=ticker,
            x=foreigner.index, y=foreigner['외국인보유비중'],
            showlegend=False,
            xhoverformat='%Y/%m/%d', 
            hovertemplate='%{x}<br>%{y:.2f}%<extra>보유비중</extra>'
        ), row = n // 2 + 1, col = n % 2 + 1, secondary_y=False
    )
    fig.add_trace(
        trace = go.Scatter(
            name=ticker,
            x=foreigner.index, y=foreigner['종가'],
            line=dict(color='#000000', dash='dot', width=1),
            showlegend=False,
            xhoverformat='%Y/%m/%d', 
            hovertemplate='%{x}<br>%{y:,d}원<extra>종가</extra>'
        ), row = n // 2 + 1, col = n % 2 + 1, secondary_y=True
    )
    

add_recession(fig)
fig.update_layout(layout)
fig.update_layout(    
    title=f'<b>은행주</b> 외국인 보유비중',
    xaxis=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis2=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis3=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis4=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    yaxis=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis2=dict(title='[KRW]', showgrid=False),
    yaxis3=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis4=dict(title='[KRW]', showgrid=False),
    yaxis5=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis6=dict(title='[KRW]', showgrid=False),
    yaxis7=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis8=dict(title='[KRW]', showgrid=False),
)

for n, annotation in enumerate(fig['layout']['annotations']):
    annotation['x'] = 0 + 0.55 * (n % 2)
    annotation['xanchor'] = 'center'
    annotation['xref'] = 'paper'
fig.show()

* 습작 노트

In [ ]:
# 금리 정보
kr_interests = macro.data.ecos('817Y002')
print(f"한국은행 금리정보:\n{kr_interests.columns.to_list()}\n")

In [ ]:
deposit = kr_interests['자금조정 예금금리'].dropna().astype(float)
# loan = kr_interests['자금조정 대출금리'].dropna().astype(float)
loan = kr_interests['정부대출금금리'].dropna().astype(float)
margin = loan - deposit

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=deposit.index,
        y=deposit,
        name='예금금리',
    )
)
fig.add_trace(
    go.Scatter(
        x=loan.index,
        y=loan,
        name='대출금리'
    )
)
fig.add_trace(
    go.Scatter(
        x=margin.index,
        y=margin,
        name='예대마진'
    )
)
